In [48]:
import os
import os.path as osp
import sys
sys.path.append("..")

import numpy as np
import open3d as o3d
from plyfile import PlyData, PlyElement

import utils

In [49]:
pcd_path = "../data/kinth/pointcloud/1720509720.141261339.pcd"
pcd = o3d.io.read_point_cloud(pcd_path)
# pcd = pcd.voxel_down_sample(voxel_size=0.01)
pcd = pcd.remove_radius_outlier(nb_points=25, radius=0.2)[0] # (pcd, new indexed from old)

points = np.asarray(pcd.points)
print("max, min, avg:", points.max(axis=0), points.min(axis=0), points.mean(axis=0))
print(f"before voxelized: {len(points)}")
points, _, _, _ = utils.voxel_downsample(points, 0.02, use_avg=False)
print(f"after voxelized: {len(points)}")
print("max, min, avg:", points.max(axis=0), points.min(axis=0), points.mean(axis=0))


max, min, avg: [4.49300003 2.94000006 2.71399999] [-3.57399988 -4.28200006 -0.296     ] [-0.21842872 -0.28041949  0.74502877]
before voxelized: 12234
after voxelized: 7677
max, min, avg: [8.07 7.23 3.01] [0.01 0.01 0.01] [3.25279015 3.75302462 1.39481178]


# draw box

In [50]:
def aabb_draw_meta(data):
    assert isinstance(data, np.ndarray) or isinstance(data, o3d.geometry.PointCloud)

    o3dpcd = None
    if isinstance(data, np.ndarray):
        assert len(data.shape) == 2
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(data)
        o3dpcd = pcd
    else:
        o3dpcd = data
    
    aabb = o3dpcd.get_axis_aligned_bounding_box()
    # print(aabb.get_min_bound(), aabb.get_max_bound())
    bbox_vtx_list = np.asarray(aabb.get_box_points())

    center = bbox_vtx_list.mean(axis=0)

    bbox_vtx_list = bbox_vtx_list - center

    draw_sequence = [
        [0, 1],
        [1, 3],
        [3, 2],
        [2, 0],
        [4, 5],
        [5, 7],
        [7, 6],
        [6, 4]
    ]

    selection_base = np.hstack([np.abs(bbox_vtx_list[0]).reshape(-1, 1), -np.abs(bbox_vtx_list[0]).reshape(-1, 1)])

    bbox_points = []
    for i in range(8):
        idx = [int(x) for x in bin(i)[2:].zfill(3)]
        bbox_points.append([
            selection_base[0][idx[0]],
            selection_base[1][idx[1]],
            selection_base[2][idx[2]]
        ])
    
    return np.array(bbox_points + center), np.array(draw_sequence)

In [51]:
bbox_points, draw_sequence = aabb_draw_meta(points)

ply_vtx = PlyElement.describe(
    data=np.core.records.fromarrays(
        np.concatenate([points, bbox_points], axis=0).transpose(),
        dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4')]
    ),
    name="vertex"
)

ply_edg = PlyElement.describe(
    data=np.core.records.fromarrays(
        (draw_sequence + len(points)).transpose(),
        dtype=[('vertex1', 'u4'), ('vertex2', 'u4')]
    ),
    name="edge"
)

ply_data = PlyData([ply_vtx, ply_edg], text=False)
ply_data.write("../data/output/bbox_test.ply")